# Wyrażenia regularne — ciąg dalszy

Dzisiaj uzupełnimy wiadomości o wyrażeniach regularnych. Poznamy:
- Zakresy znaków z minusem wewnątrz zbiorów znaków objętych nawiasami kwadratowymi `[...]`
- Zanegowane zbiory znaków objęte nawiasami kwadratowymi z daszkiem `[^...]`
- Pytajnik `?`. Pytajnik za dowolnym atomem oznacza 0 lub 1 dopasowanie tego atomu.
- Plus `+`. Plus za dowolnym atomem oznacza 1 lub więcej dopasowań tego atomu.

Ponadto oprócz metody `search` omówionej ostatnio, poznamy:
- Zapamiętywanie dopasowanych fragmentów tekstu w **grupach**.
- Metodę `sub`. Metoda `sub` zastępuje danym łańcuchem znaków wszystkie wystąpienia wzorca, które znalazła w innym łańcuchu znaków.

Po kolei:

In [ ]:
import re

**1.** Wewnątrz nawiasów kwadratowych umieszczamy **zbiory znaków**, na przykład:

In [ ]:
ZASADA_DNA_RE = re.compile(r'[ACGT]')

Wewnątrz nawiasów kwadratowych możemy też umieszczać **zakresy znaków**. Zakres znaków składa się ze:
- pierwszego znaku zakresu
- znaku minusa `-`
- ostatniego znaku zakresu

Na przykład:

In [ ]:
MAŁA_LITERA_ANGIELSKA = re.compile(r'[a-z]')
LITERA_ANGIELSKA = re.compile(r'[a-zA-Z]')

Ponieważ kody polskich liter nie mają wiele wspólnego z kolejnością alfabetyczną przyjętą w języku polskim, w zbiorach znaków trzeba je wymieniać pojedynczo, na przykład:

In [ ]:
LITERA = re.compile(r'[a-ząćęłńóśźżA-ZĄĆĘŁŃÓŚŹŻ]')

**2.** **Zanegowane zbiory znaków** pozwalają dopasować dowolny pojedynczy znak, który nie należy do danego zbioru. Aby zanegować zbiór znaków, umieszczamy zaraz za otwierającym nawiasem kwadratowym znak daszka `^`. Na przykład:

In [ ]:
NIELITERA = re.compile(r'[^a-ząćęłńóśźżA-ZĄĆĘŁŃÓŚŹŻ]')

**3.** Znamy **gwiazdkę Kleene'a** `*`. Gwiazdka za dowolnym atomem oznacza 0 lub więcej dopasowań tego atomu.
Podobnie działa pytajnik `?` i plus `+`.

**Pytajnik** oznacza 0 lub 1 dopasowanie poprzedniego atomu, na przykład:

In [ ]:
# Regexp USER_RE pasuje do napisu 'user' i do napisu 'users'
USER_RE = re.compile(r'^users?$')
print(USER_RE.search('user'))
print(USER_RE.search('users'))
print(USER_RE.search('foo'))

In [ ]:
# Regexp LESS_THAN_100 pasuje do takich łańcuchów, które odpowiadają liczbom od 1 do 99
LESS_THAN_100 = re.compile(r'^[1-9][0-9]?$')
print(LESS_THAN_100.search('4'))
print(LESS_THAN_100.search('44'))
print(LESS_THAN_100.search('444'))

**4.** **Plus** `+` oznacza 1 lub więcej dopasowanie poprzedniego atomu, na przykład:

In [ ]:
# Regexp LICZBA_RE pasuje do liczb naturalnych
LICZBA_RE = re.compile(r'^[0-9]+$')
print(LICZBA_RE.search('2025'))
print(LICZBA_RE.search(''))
print(LICZBA_RE.search('foo'))

**5.** Atomy otoczone nawiasami okrągłymi `(...)` tworzą **grupy**. Dzięki grupom możemy wyodrębniać
fragmenty tekstu. Grupy są numerowane od 1 wzwyż. Korzystamy z nich tak:
- Przypisujemy wynik metody `search` do zmiennej
- Jeśli ta zmienna ma wartość `None`, to znaczy, że wyrażenie regularne się nie dopasowało
- Jeśli ta zmienna ma wartość inną niż `None`, to jej pola `.group(1)`, `.group(2)` itd. są kolejnymi grupami.

Oto przykład użycia grup:

In [ ]:
# NUMER_TELEFONU_RE pasuje do numeru telefonu w formacie 'XX XXX-XXXX'
NUMER_TELEFONU_RE = re.compile(r'(^[0-9][0-9]) ([0-9][0-9][0-9]-[0-9][0-9][0-9][0-9])$')
m = NUMER_TELEFONU_RE.search('12 345-6789')
if m is None:
    print('To nie jest numer stacjonarny')
else:
    print(f'Kierunkowy: {m.group(1)}, lokalny: {m.group(2)}')

**6.** Metoda `sub` zastępuje danym łańcuchem znaków wszystkie wystąpienia wzorca, które znalazła w innym łańcuchu znaków. W tym łańcuchu, którym zastępuje wystąpienia wzorca, możemy używać skrótów `\1`, `\2` itd., które oznaczają kolejne grupy. Lepiej pamiętać o przedrostku `r` przed tym łańcuchem, żeby znak `\` nie był znakiem specjalnym, tylko oznaczał lewy ukośnik.

Przykłady użycia metody `sub`:

In [ ]:
DATA_DD_MM_RRRR_RE = re.compile(r'^([0-3][0-9])-([01][0-9])-([0-9][0-9][0-9][0-9])$')
print(DATA_DD_MM_RRRR_RE.sub(r'\3-\2-\1', '01-10-2025'))

In [ ]:
BRZYDKIE_SŁOWA_RE = re.compile(r'(lipa|chała|kiepsk.)')
recenzja = 'Ten film jest kiepski. Zupełna lipa.'
print(BRZYDKIE_SŁOWA_RE.sub('****', recenzja))

## Zadania na czwórkę

Celem dzisiejszych zadań jest opracowanie programu, który odpowiada na pytania o pracowników
Wydziału Bezpieczeństwa i Informatyki, czerpiąc odpowiedzi z niewielkiej bazy danych.
Program powinien sensownie odpowiadać np. na takie pytania:
```
    W której katedrze pracuje doktor Ciura?
    Który pokój ma pani dyrektor?
    Jacy profesorowie są pracownikami Katedry Inżynierii Komputerowej?
    Kto ma telefon o numerze 78-43?
    Czy znasz kogoś o imieniu Anna?
    Parlez-vous français ?
```

Oczywiście gdzieś trzeba postawić granicę, bo od prostego programu nie można się spodziewać
zadowalających odpowiedzi na wszelkie możliwe pytania:
```
    Jakie było najgłupsze pytanie, które ci kiedykolwiek zadano?
```

Sztuka polega na rozsądnym odpowiadaniu na jak najwięcej pytań bez nadmiernego komplikowania programu.
Zakładamy, że użytkownik programu ma niewielkie doświadczenie z komputerami
i nie chce czytać podręcznika ani uczyć się reguł zadawania pytań.
Wiemy o nim tylko to, że zna język polski i umie pisać na klawiaturze.

Nie należy zakładać, że użytkownik będzie się starał wywieść program w pole.
Dialog między użytkownikiem a programem można raczej porównać do próby porozumienia się w języku,
który słabo rozumiemy.
Ludzie w takich sytuacjach zawsze znajdą sposób, żeby przekazać sobie, o co chodzi.

**Zadanie 1**. Napisz funkcję `podziel_na_wyrazy(pytanie: str) -> list`. Funkcja `podziel_na_wyrazy` ma:
- zastąpić spacją każdy znak łańcucha `pytanie`, które nie jest literą ani cyfrą, korzystając z metody `sub`
- podzielić otrzymany łańcuch na wyrazy metodą `.split`
- zwrócić otrzymaną listę łańcuchów

In [ ]:
ANI_LITERA_ANI_CYFRA_RE = re.compile(r'[^a-ząćęłńóśźżA-ZĄĆĘŁŃÓŚŹŻ0-9]')

def podziel_na_wyrazy(pytanie: str) -> list:
    """Usuwa z pytania nielitery i niecyfry, dzieli je na wyrazy"""

In [ ]:
w = podziel_na_wyrazy('Gdzie pracuje dr Kowalska-Wójcik?')
assert w == ['Gdzie', 'pracuje', 'dr', 'Kowalska', 'Wójcik'], w

Oto pomocnicza funkcja, która zamienia duże litery na małe, a potem zastępuje
litery z polskimi znakami diakrytycznymi literami bez znaków diakrytycznych.
Nie trzeba z nią nic robić.

In [ ]:
BEZ_ZNAKÓW_DIAKRYTYCZNYCH = {
    'ą': 'a',
    'ć': 'c',
    'ę': 'e',
    'ł': 'l',
    'ń': 'n',
    'ó': 'o',
    'ś': 's',
    'ź': 'z',
    'ż': 'z',
}

def bez_znaków_diakrytycznych(wyraz: str) -> str:
    wyraz = wyraz.lower()
    return ''.join(BEZ_ZNAKÓW_DIAKRYTYCZNYCH.get(c, c) for c in wyraz)

In [ ]:
assert bez_znaków_diakrytycznych('Żabińska') == 'zabinska'

**Zadanie 2**. Oto funkcja, która obcina końcówki odmiany rzeczownika lub przymiotnika.
Pomocny prowadzący już w niej uwzględnił nietypowe przypadki.
Proszę uzupełnić tylko regexp `BEZ_DŁUŻSZEJ_KOŃCÓWKI`,
który zawiera regularne końcówki odmiany rzeczowników i przymiotników.

In [ ]:
# Miejscownik liczby pojedynczej rodzaju męskiego.
BEZ_KOŃCÓWKI_Z_WYMIANĄ_CI_T = re.compile('^(.+)cie$')
BEZ_KOŃCÓWKI_Z_WYMIANĄ_DZI_D = re.compile('^(.+d)zie$')
# Celownik i miejscownik liczby pojedynczej rodzaju żeńskiego.
BEZ_KOŃCÓWKI_Z_WYMIANĄ_CE_K = re.compile('^(.+)ce$')
# Miejscownik l.p. r.męskiego i mianownik liczby mnogiej rodzaju męskiego.
BEZ_KOŃCÓWKI_Z_WYMIANĄ_RZ_R = re.compile('^(.+r)z[ey]$')

# Nazwiska zakończone na -cień, -dziec, -dzień, -rzec, -rzeł.
BEZ_KOŃCÓWKI_Z_WYMIANĄ_CIE_T = re.compile('^(.+)cie([nń])$')
BEZ_KOŃCÓWKI_Z_WYMIANĄ_DZIE_D = re.compile('^(.+d)zie([cnń])$')
BEZ_KOŃCÓWKI_Z_WYMIANĄ_RZE_R = re.compile('^(.+r)ze([clł])$')

# Dwusylabowe i dłuższe nazwiska zakończone na -ie[ćlłńr].
BEZ_RUCHOMEGO_IE = re.compile('^(.*[aąeęioóuy].+)ie([cćlłnńr])$')
# Nazwiska i imiona zakończone na -e[ćlłńr].
BEZ_RUCHOMEGO_E = re.compile('^(.+[^i])e([cćlłnńr])$')
# Dwusylabowe i dłuższe nazwiska zakończone na -ie[ck].
BEZ_RUCHOMEGO_IEC_IEK = re.compile('^(.*[aąeęioóuy].+)ie([ck])$')
# Dwusylabowe i dłuższe nazwiska zakończone na -e[ck].
BEZ_RUCHOMEGO_EC_EK = re.compile('^(.*[aąeęioóuy].*[^i])e([ck])$')

# Regularne końcówki odmiany rzeczowników i przymiotników,
# oprócz końcówek złożonych z samych samogłosek.
# Proszę uzupełnić ten regexp.
BEZ_DŁUŻSZEJ_KOŃCÓWKI = re.compile('^(.+)(ego|emu)$')

def bez_końcówki(wyraz: str) -> list:
    """Zwraca wyraz bez końcówki odmiany rzeczownika lub przymiotnika"""
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_CI_T.match(wyraz);
    if m: return[m.group(1) + 't']
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_DZI_D.match(wyraz)
    if m: return[m.group(1)]
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_CE_K.match(wyraz)
    if m: return[m.group(1) + 'k']
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_RZ_R.match(wyraz)
    if m: return[m.group(1), m.group(1) + 'z']
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_CIE_T.match(wyraz)
    if m: return[wyraz, m.group(1) + 't' + m.group(2)]
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_DZIE_D.match(wyraz)
    if m: return[wyraz, m.group(1) + m.group(2)]
    m = BEZ_KOŃCÓWKI_Z_WYMIANĄ_RZE_R.match(wyraz)
    if m: return[wyraz,
                 m.group(1) + 'z' + m.group(2),
                 m.group(1) + m.group(2)]
    m = BEZ_RUCHOMEGO_IE.match(wyraz)
    if m: return [wyraz, m.group(1) + m.group(2)]
    m = BEZ_RUCHOMEGO_E.match(wyraz)
    if m: return [wyraz, m.group(1) + m.group(2)]
    m = BEZ_RUCHOMEGO_IEC_IEK.match(wyraz)
    if m: return[m.group(1) + m.group(2)]
    m = BEZ_RUCHOMEGO_EC_EK.match(wyraz)
    if m: return[m.group(1) + m.group(2)]
    if wyraz == 'nie':
        return [wyraz]
    m = BEZ_DŁUŻSZEJ_KOŃCÓWKI.match(wyraz)
    if m: wyraz = m.group(1)
    return [wyraz.rstrip('aąeęiouy')]

Proszę przetestować zmiany regexpu `BEZ_DŁUŻSZEJ_KOŃCÓWKI` poniżej:

In [ ]:
def podziel_i_odetnij_końcówki(pytanie: str) -> list:
    """Dzieli pytanie na wyrazy i odcina ich końcówki"""
    wynik = []
    for wyraz in podziel_na_wyrazy(pytanie):
        wyraz = bez_znaków_diakrytycznych(wyraz)
        wynik.append(bez_końcówki(wyraz))
    return wynik

In [ ]:
w = podziel_i_odetnij_końcówki('Kowalski Kowalskiego Kowalskiemu Kowalskim Kowalska Kowalskiej Kowalską')
assert w == [['kowalsk'], ['kowalsk'], ['kowalsk'], ['kowalsk'], ['kowalsk'], ['kowalsk'], ['kowalsk']], w
w = podziel_i_odetnij_końcówki('Szczęsny Szczęsnego Szczęsnemu Szczęsnym Szczęsna Szczęsnej Szczęsną')
assert w == [['szczesn'], ['szczesn'], ['szczesn'], ['szczesn'], ['szczesn'], ['szczesn'], ['szczesn']], w
w = podziel_i_odetnij_końcówki('Nowak Nowaka Nowakowi Nowakiem Nowaku')
assert w == [['nowak'], ['nowak'], ['nowak'], ['nowak'], ['nowak']], w
w = podziel_i_odetnij_końcówki('doktor doktora doktorowi doktorem')
assert w == [['doktor'], ['doktor'], ['doktor'], ['doktor']], w
w = podziel_i_odetnij_końcówki('profesorowie profesorów profesorom profesorami profesorach')
assert w == [['profesor'], ['profesor'], ['profesor'], ['profesor'], ['profesor']], w
w = podziel_i_odetnij_końcówki('doktorze doktorzy Jerzy Zygmuncie Dawidzie Magdzie jednostce')
assert w == [['doktor', 'doktorz'], ['doktor', 'doktorz'], ['jer', 'jerz'], ['zygmunt'], ['dawid'], ['magd'], ['jednostk']], w
w = podziel_i_odetnij_końcówki('Marzec Kwiecień Grudzień')
assert w == [['marzec', 'marzc', 'marc'], ['kwiecien', 'kwietn'], ['grudzien', 'grudn']], w
w = podziel_i_odetnij_końcówki('Niemiec Niemca Bieniek Bieńka')
assert w == [['niemiec', 'niemc'], ['niemc'], ['bienk'], ['bienk']], w
w = podziel_i_odetnij_końcówki('Pawelec Pawelca Dudek Dudka')
assert w == [['pawelec', 'pawelc'], ['pawelc'], ['dudk'], ['dudk']], w
w = podziel_i_odetnij_końcówki('Dec Piec Ćwiek Skrzek')
assert w == [['dec'], ['piec'], ['cwiek'], ['skrzek']], w
w = podziel_i_odetnij_końcówki('Dziegieć Grygiel Szczygieł Stępień Węgier')
assert w == [['dziegiec', 'dziegc'], ['grygiel', 'grygl'], ['szczygiel', 'szczygl'], ['stepien', 'stepn'], ['wegier', 'wegr']], w
w = podziel_i_odetnij_końcówki('Kopeć Wróbel Gaweł Styczeń Majcher')
assert w == [['kopec', 'kopc'], ['wrobel', 'wrobl'], ['gawel', 'gawl'], ['styczen', 'styczn'], ['majcher', 'majchr']], w
w = podziel_i_odetnij_końcówki('Kmieć Chmiel Kieł Bień Sier')
assert w == [['kmiec'], ['chmiel'], ['kiel'], ['bien'], ['sier']], w
w = podziel_i_odetnij_końcówki('Maria Marii')
assert w == [['mar'], ['mar']], w
w = podziel_i_odetnij_końcówki('nie')
assert w == [['nie']], w

Dane o pracownikach są zapisane w pliku `pracownicy.csv`. Format tego pliku to CSV, czyli
comma-separated values, czyli wartości oddzielone przecinkami. Wartości w tym pliku to kolejno:
- jednostka organizacyjna
- dane pracownika
- numer telefonu pracownika
- numer pokoju pracownika

Oto początek tego pliku:
```
Katedra Fizyki i Matematyki Stosowanej,"dr inż. Magdalena Krupska-Klimczak, prof. UKEN, Kierownik Katedry",7847,406N
Katedra Fizyki i Matematyki Stosowanej,"dr hab. Piotr Czerski, prof. UKEN",7847,406N
Katedra Fizyki i Matematyki Stosowanej,"dr hab. Tomasz Dobrowolski, prof. UKEN",7867,307N
Katedra Fizyki i Matematyki Stosowanej,dr Leszek Głowacki,7848,405N
```

Poniższa funkcja odczytuje dane z pliku `pracownicy.csv` i zapisuje je w bazie danych
`pracownicy.sqlite`. Ta funkcja zawiera fragmenty kodu w języku SQL.

In [ ]:
import csv
import os
import sqlite3

BAZA_DANYCH = 'pracownicy.sqlite'

def utwórz_bazę_danych():
    try:
        os.remove(BAZA_DANYCH)
    except OSError:
        pass
    connection = sqlite3.connect(BAZA_DANYCH)
    cursor = connection.cursor()
    cursor.execute(
        """
        CREATE TABLE Pracownicy(
            docid INTEGER PRIMARY KEY,
            jednostka TEXT,
            pracownik TEXT,
            telefon TEXT,
            pokój TEXT
        )
        """)
    cursor.execute(
        """
        CREATE VIRTUAL TABLE PracownicyFTS USING fts4(dane)
        """)
    with open('pracownicy.csv', 'rt', encoding='utf-8') as pracownicy:
        for jednostka, pracownik, telefon, pokój in csv.reader(pracownicy):
            cursor.execute(
                """
                INSERT INTO Pracownicy(jednostka, pracownik, telefon, pokój)
                VALUES (?,?,?,?)
                """,
                (jednostka, pracownik, telefon, pokój))
            docid = cursor.lastrowid
            jednostka_fts = sum(podziel_i_odetnij_końcówki(jednostka), [])
            pracownik_fts = sum(podziel_i_odetnij_końcówki(pracownik), [])
            cursor.execute(
                """
                INSERT INTO PracownicyFTS(docid, dane)
                VALUES (?,?)
                """,
                (docid, ' '.join(jednostka_fts + pracownik_fts + [telefon, pokój.lower()])))
    connection.commit()
    connection.close()

In [ ]:
utwórz_bazę_danych()

Do wyszukiwania danych w tabeli `Pracownicy` służy wirtualna tabela `PracownicyFTS`
(FTS = Full-Text Search). Zawiera ona pozbawione końcówek odmiany i polskich znaków diakrytycznych
oraz sprowadzone do małych liter wyrazy, służące do wyszukiwania.

Na przykład temu wierszowi tabeli `Pracownicy`:
```
INSERT INTO Pracownicy(docid, jednostka, pracownik, telefon, pokój)
VALUES (77, 'Katedra Inżynierii Oprogramowania', 'dr inż. Marcin Ciura', '7854', '411N');
```
odpowiada następujący wiersz tabeli `PracownicyFTS`:
```
INSERT INTO PracownicyFTS(docid, dane)
VALUES(77, 'katedr inzynier oprogramowan dr inz marcin ciur 7854 411n');
```
Olbrzymia większość pytań do naszego programu zawiera:
- określenie ***wartości*** pewnych kolumn
- być może wyszczególnienie innych **kolumn**, których wartość interesuje użytkownika
- oraz „szum”, czyli wyrazy, które nie wpływają na interpretację pytania, na przykład:

W której **katedrze** pracuje ***doktor Ciura***?

Który **pokój** ma pani ***dyrektor***?

Jacy ***profesorowie*** są pracownikami ***Katedry Inżynierii Komputerowej***?

Kto ma **telefon** o numerze ***78-43***?

Czy znasz kogoś o imieniu ***Anna***?

Powyższym pytaniom odpowiadają następujące zapytania SQL:
```
    SELECT pracownik, jednostka
    FROM Pracownicy JOIN PracownicyFTS USING(docid)
    WHERE PracownicyFTS MATCH 'dr ciur';

    SELECT pracownik, pokój
    FROM Pracownicy JOIN PracownicyFTS USING(docid)
    WHERE PracownicyFTS MATCH 'dyrektor';

    SELECT pracownik, jednostka
    FROM Pracownicy JOIN PracownicyFTS USING(docid)
    WHERE PracownicyFTS MATCH 'prof katedr inzynier komputerow';

    SELECT pracownik, telefon
    FROM Pracownicy JOIN PracownicyFTS USING(docid)
    WHERE PracownicyFTS MATCH '7843';

    SELECT pracownik
    FROM Pracownicy JOIN PracownicyFTS USING(docid)
    WHERE PracownicyFTS MATCH 'ann';
```

**Zadanie 3**. Oto funkcja `znajdź_kolumny(wyrazy: list) -> set`. Ta funkcja wyszukuje na liście wyrazów
wyrazy kluczowe, którym odpowiadają kolumny tabeli `Pracownicy`. Proszę dodać kilka wyrazów do
słownika `KOLUMNY`. Przykład: `'katedr': JEDNOSTKA`.

In [ ]:
JEDNOSTKA = 'jednostka'
PRACOWNIK = 'pracownik'
TELEFON = 'telefon'
POKÓJ = 'pokój'

KOLUMNY = {
    'jednostk': JEDNOSTKA,
    'telefon': TELEFON,
    'pokoj': POKÓJ,
}

def znajdź_kolumny(wyrazy: list) -> set:
    """Zwraca zbiór kolumn, który odpowiada wyrazom"""
    kolumny = { PRACOWNIK }
    for wyraz in wyrazy:
        for forma in wyraz:
            if forma in KOLUMNY:
                kolumny.add(KOLUMNY[forma])
    return kolumny

In [ ]:
w = znajdź_kolumny([['w'], ['ktor'], ['katedr'], ['pracuj'], ['filip']]) # W której katedrze pracuje Filip?
assert w == { PRACOWNIK, JEDNOSTKA }, w
w = znajdź_kolumny([['jak'], ['jest'], ['numer'], ['filip']]) # Jaki jest numer Filipa?
assert w == { PRACOWNIK, TELEFON }, w
w = znajdź_kolumny([['gd'], ['jest'], ['gabinet'], ['kierownik']]) # Gdzie jest gabinet kierownika?
assert w == { PRACOWNIK, POKÓJ }, w

Oto funkcja `znajdź_warunek(wyrazy: list) -> str`. Ta funkcja wyszukuje na liście wyrazów
te wartości, które występują w tabeli `PracownicyFTS`.

In [ ]:
ZNACZĄCE_FORMY = set()
connection = sqlite3.connect(BAZA_DANYCH)
for wiersz in connection.execute('SELECT * FROM PracownicyFTS'):
    for pole in wiersz:
        for forma in pole.split():
            ZNACZĄCE_FORMY.add(forma)
print(ZNACZĄCE_FORMY)

In [ ]:
SYNONIMY = {
    'doktor': 'dr',
    'doktorat': 'dr',
    'habilitowan': 'hab',
    'habilitacj': 'hab',
    'profesor': 'prof',
}

def znajdź_warunek(wyrazy: list) -> str:
    """Zwraca tekst warunku do zapytania SQL"""
    warunek = []
    for wyraz in wyrazy:
        w = []
        for forma in wyraz:
            forma = SYNONIMY.get(forma, forma)
            if forma in ZNACZĄCE_FORMY:
                w.append(forma)
        if w:
            warunek.append('(' + ' OR '.join(w) + ')')
    return ' '.join(warunek)

In [ ]:
# Gdzie ma pokój doktor Nawalaniec?
w = znajdź_warunek([['gd'], ['m'], ['pokoj'], ['doktor'], ['nawalaniec', 'nawalanc']])
assert w == '(dr) (nawalaniec OR nawalanc)', w

Dzięki powyższym funkcjom możemy napisać ostateczną funkcję `odpowiedz(pytanie: str)`.
Argumentem tej funkcji jest pytanie w języku polskim. Funkcja `odpowiedz` wypisuje tabelę
z odpowiedzią na to pytanie.

In [ ]:
def odpowiedz(pytanie: str):
    """Odpowiada na pytanie zadane po polsku"""
    wyrazy = podziel_i_odetnij_końcówki(pytanie)
    kolumny = ', '.join(znajdź_kolumny(wyrazy))
    warunek = znajdź_warunek(wyrazy)
    zapytanie_sql = f"""
        SELECT {kolumny}
        FROM Pracownicy JOIN PracownicyFTS USING(docid)
        WHERE PracownicyFTS MATCH ?
    """
    print(zapytanie_sql)  # Można zakomentować tę linię
    print(warunek)  # Tę linię też
    for wiersz in connection.execute(zapytanie_sql, [warunek]):
        print('\t'.join(wiersz))

In [ ]:
odpowiedz('W której jednostce pracuje Wojciech i jaki ma numer telefonu?')

Zadaj kilka pytań:

In [ ]:
while True:
    pytanie = input('> ')
    if not pytanie:
        break
    odpowiedz(pytanie)

## Zadanie na piątkę

**Zadanie 4**. Proszę napisać regexp, który normalizuje numery telefonów.
Wszystkie numery telefonów na UKEN zaczynają się od `+48 12 662`.
Należy uzupełnić regexp `TELEFON_RE` tak, aby działanie `(grupa 1) + (grupa 2)` zwracało
czterocyfrową końcówkę numeru telefonu w następujących przypadkach:
- Numer typu `78-99`, pasujący do regexpu `r'([0-9][0-9])[ -]*([0-9][0-9])'`
- Numer typu `662-78-99`, pasujący do regexpu `r'662[ -]*([0-9][0-9])[ -]*([0-9][0-9])'`
- Numer typu `12-662-78-99`, pasujący do regexpu `r'12[ -]*662[ -]*([0-9][0-9])[ -]*([0-9][0-9])'`
- Numer typu `+48 12-662-78-99`, pasujący do regexpu `r'\+48[ -]*12[ -]*662[ -]*([0-9][0-9])[ -]*([0-9][0-9])'`

**Wskazówki**:
- Regexp `[ -]*` pasuje do ciągu spacji lub znaków minusa.
- `(?:...)` tworzy taką grupę, której nie zostanie nadany numer.

In [ ]:
TELEFON_RE = re.compile(
    r'Tu wpisać regexp')

def normalizuj_numery_telefonów(tekst: str) -> str:
    """Zwraca `tekst` ze znormalizowanymi numerami telefonów"""
    return TELEFON_RE.sub(r'\1\2', tekst)

In [ ]:
w = normalizuj_numery_telefonów('21 04 21-05 662 - 21 - 06 126622107 +48 12 662 21 08')
assert w == '2104 2105 2106 2107 2108', w

## Zadanie na szóstkę

**Zadanie 5**. Do słownika `SYNONIMY` dodano kilka wyrazów, które zaprzeczają tej części warunku, która występuje zaraz za nimi.
Zmień funkcję `znajdź_warunek` tak, aby w pytaniach można było używać przeczenia.

Przyjrzyj się testom pod funkcją `znajdź_warunek` i wymyśl taki sposób tworzenia warunku, żeby
te testy przechodziły pomyślnie.

In [ ]:
SYNONIMY = {
    'doktor': 'dr',
    'doktorat': 'dr',
    'habilitowan': 'hab',
    'habilitacj': 'hab',
    'profesor': 'prof',
    'bez': 'nie',
    'brak': 'nie',
    'oprocz': 'nie',
    'poz': 'nie',  # "poza"
    'an': 'nie',  # "ani"
}

def znajdź_warunek(wyrazy: list) -> str:
    """Zwraca tekst warunku do zapytania SQL"""
    warunek = []
    for wyraz in wyrazy:
        w = []
        for forma in wyraz:
            forma = SYNONIMY.get(forma, forma)
            if forma in ZNACZĄCE_FORMY:
                w.append(forma)
        if w:
            warunek.append('(' + ' OR '.join(w) + ')')
    return ' '.join(warunek)

In [ ]:
# Jaki jest Wojciech Nawalaniec?
w = znajdź_warunek([['jak'], ['jest'], ['wojciech'], ['nawalaniec', 'nawalanc']])
assert w == '(wojciech) (nawalaniec OR nawalanc)', w
# Jaki jest Wojciech, ale nie Nawalaniec?
w = znajdź_warunek([['jak'], ['jest'], ['wojciech'], ['al'], ['nie'], ['nawalaniec', 'nawalanc']])
assert w == '(wojciech) NOT (nawalaniec OR nawalanc)', w

# Podaj doktorów z habilitacją.
w = znajdź_warunek([['podaj'], ['doktor'], ['z'], ['habilitacj']])
assert w == '(dr) (hab)', w
# Podaj doktorów bez habilitacji.
w = znajdź_warunek([['podaj'], ['doktor'], ['bez'], ['habilitacj']])
assert w == '(dr) NOT (hab)', w

Zadaj kilka pytań z przeczeniami:

In [ ]:
while True:
    pytanie = input('> ')
    if not pytanie:
        break
    odpowiedz(pytanie)

Opracowano na podstawie zadania 5. z seminarium programowania i rozwiązywania zadań,
prowadzonego w 1981 roku na Uniwersytecie Stanforda przez Donalda E. Knutha
i Josepha S. Weeninga: [http://i.stanford.edu/pub/cstr/reports/cs/tr/83/989/CS-TR-83-989.pdf#page=71].